In [1]:
from pathlib import Path
import os
SRC=os.path.join(Path.cwd().parent, 'src')
import sys
sys.path.append(SRC)
from Topology import TopologyGenerator
from ChromatinDynamics import ChromatinDynamics
import openmm.unit as unit
import numpy as np
from LoopExtrusion import Loop_Extruders, Extrusion_kinetics
from Reporters import save_pdb
import h5py

In [2]:
generator = TopologyGenerator()
generator.gen_top([100])


In [3]:
for chain in generator.topology.chains():
    print(list(chain.atoms())[0].id, type(list(chain.atoms())[-1].id))

0 <class 'str'>


In [4]:
sim3D = ChromatinDynamics(generator.topology, 'LE', output_dir=f'{Path.cwd()}/output', console_stream=True, platform_name="OpenCL")

2025-04-28 15:44:06,788 | INFO | ChromatinDynamics | ************************************************************
2025-04-28 15:44:06,789 | INFO | ChromatinDynamics |                      Chromatin Dynamics                     
2025-04-28 15:44:06,790 | INFO | ChromatinDynamics | ************************************************************
2025-04-28 15:44:06,790 | INFO | ChromatinDynamics | Storing output in /Users/sb95/Library/CloudStorage/Box-Box/Research_current/ChromatinDynamics/notebooks/output
2025-04-28 15:44:06,791 | INFO | ChromatinDynamics | System initialized with 100 particles.
2025-04-28 15:44:06,792 | INFO | Platforms | Platform 'OpenCL' is available and selected.
2025-04-28 15:44:06,792 | INFO | ChromatinDynamics | force_field_manager initialized. Use this to add forces to the system before setting up simulation.
2025-04-28 15:44:06,793 | INFO | ChromatinDynamics | ------------------------------------------------------------


In [5]:
sim3D.force_field_manager.add_harmonic_bonds()
# sim.force_field_manager.removeCOM()
sim3D.force_field_manager.add_self_avoidance()

2025-04-28 15:44:07,689 | INFO | Forcefield | Adding 99 harmonic bonds with parameters:
2025-04-28 15:44:07,690 | INFO | Forcefield | length: 1.0, spring constant (k): 10.0, group: 0
2025-04-28 15:44:07,691 | INFO | Forcefield | HarmonicBonds force successfully added to system.
2025-04-28 15:44:07,691 | INFO | Forcefield | --------------------------------------------------
2025-04-28 15:44:07,692 | INFO | Forcefield | Adding Self-avoidance force with parameters:
2025-04-28 15:44:07,692 | INFO | Forcefield | Ecut=4.0, k_rep=5.0, r_rep=1.0, cutoff=5.0, group=2
2025-04-28 15:44:07,693 | INFO | Forcefield | Added exclusions from bonded monomers.
2025-04-28 15:44:07,693 | INFO | Forcefield | SelfAvoidance force successfully added to system.
2025-04-28 15:44:07,693 | INFO | Forcefield | --------------------------------------------------


In [6]:
sim3D.simulation_setup(init_struct='randomwalk',save_pos=True, save_energy=True)

2025-04-28 15:44:08,558 | INFO | ChromatinDynamics | ------------------------------------------------------------
2025-04-28 15:44:08,560 | INFO | Integrators | Valid integrators: ['langevin', 'brownian', 'active-brownian']| Selected: langevin
2025-04-28 15:44:08,561 | INFO | Integrators | LangevinIntegrator: temperatute=120.0 | friction=0.1 | timestep=0.01
2025-04-28 15:44:08,653 | INFO | ChromatinDynamics | Setting up context...
2025-04-28 15:44:08,654 | INFO | Utilities | Random walk created. Position shape: (100, 3)
2025-04-28 15:44:08,656 | INFO | ChromatinDynamics | Simulation set up complete!
2025-04-28 15:44:08,656 | INFO | ChromatinDynamics | ------------------------------------------------------------------------------------------------------------------------
2025-04-28 15:44:08,657 | INFO | ChromatinDynamics | Index  Force Class                    Force Name           Group    Particles    Bonds        Exclusions   P.E./Particle       
2025-04-28 15:44:08,657 | INFO | Chrom

In [7]:
sim3D.run(1000)

2025-04-28 15:44:10,228 | INFO | ChromatinDynamics | ------------------------------------------------------------
2025-04-28 15:44:10,229 | INFO | ChromatinDynamics | Running simulation for 1000 steps...
2025-04-28 15:44:10,356 | INFO | ChromatinDynamics | Completed 1000 steps in 0.13s (7917 steps/s)
2025-04-28 15:44:10,356 | INFO | ChromatinDynamics | ------------------------------------------------------------


7917.022630020216

In [9]:
sim3D.force_field_manager.removeForce(sim3D.simulation.context,'HarmonicBonds')

2025-04-28 15:45:47,683 | INFO | Forcefield | Removed HarmonicBonds from the system!


In [7]:
lef = Loop_Extruders(name='smc',num_LE=1, chain_topology=generator.topology,
                     k_off=0.01, k_hop=0.1, kstep_LE=1.0, stepsize_LE=3.0,
                     sim3D=sim3D, tau=1e2)
print("3D hopping", lef.hop3D)
print("Preferential loading", lef.preferential_LE_loading)
print("Temperature", lef.temp)
# lef.update_immobile_blockers(boundaries)
lef.unbind_at_end = True

Random seed 36394945
Randomly distributing the LEFs
3D hopping False
Preferential loading 0.0
Temperature 0.0


In [8]:
sim1D = Extrusion_kinetics(lef)

Initialized extrusion kinetics with the random seed  32812


In [9]:

def closest_lower_index(t, A):
    return max((i for i, x in enumerate(A) if x <= t), key=lambda i: A[i], default=None)

def add_new_LEFBonds(sim1D, sim3D, nsteps):
    anchor_locs = []
    activation_times = []
    leids = []
    for time_val in sorted(sim1D.lef.anchor_loc_dict.keys())[-nsteps-1:]:
        
        anchor_locs.append(sim1D.lef.anchor_loc_dict[time_val][0])
        activation_times.append(time_val)
        leids.append(sim1D.lef.anchor_loc_dict[time_val][1])
        # print(time_val,sim1D.lef.anchor_loc_dict[time_val] ,"yosemite")
    
    anchor_locs = np.array(anchor_locs, dtype=int)
    leids = np.array(leids, dtype=str)
    unique_anchor_locs = list(map(list, set(map(tuple, anchor_locs.reshape(-1,2)))))
    
    sim3D.force_field_manager.removeForce("LEFBonds")
    sim3D.force_field_manager.addLEFBonds(anchors=unique_anchor_locs)
    
    # print(f"Total LEF bonds added: {sim3D.forceDict['LEFBond'].getNumBonds()}")
    # for bid in range(sim3D.forceDict["LEFBond"].getNumBonds()):
    #     params=sim3D.forceDict['LEFBond'].getBondParameters(bid)
    #     print('bond params', params)
    
    return anchor_locs, activation_times, leids
        
def activate_LEFBonds(sim3D, activated_anchors,kLE):
    
    num_LEFBonds = sim3D.forceDict["LEFBond"].getNumBonds()
    for bond_id in range(num_LEFBonds):
        particle1, particle2, _ = sim3D.forceDict['LEFBond'].getBondParameters(bond_id)
        k_LE_bond = 0.0
        for anchor1, anchor2 in activated_anchors:
            
            if anchor1==particle1 and anchor2==particle2: 
                # print('grand canyon', anchor1, anchor2, particle1, particle2)
                k_LE_bond = kLE
                
        sim3D.forceDict['LEFBond'].setBondParameters(bond_id, particle1, particle2, 1.0, k_LE_bond)
    sim3D.forceDict['LEFBond'].updateParametersInContext(sim3D.context)

def run_sim(sim3D, sim1D, lef, tstop, update_interval=10.0, verbose=False):

    anchor_loc_dict = {}
    
    resisting_force = []
    active_force = [[],[]]
    # matching time scales
    t1D_unit = 1 # second
    t3D_unit = 0.1 # second
    dt=0.01 # t3D
    blocksize = 500
    tblock = blocksize*dt*t3D_unit # second

    time3D = 0
    update_lag = 10 #num events before updating 3D sims
    update_lag_time = update_interval

    # temper_polymer(sim3D, blocksize=5000, dT=50.0)
    
    num_events_since_last_3D_update = 0
    time_since_last_3D_update = 0.0

    while time3D<tstop:
        sim1D.simulate(verbose=verbose)    
        num_events_since_last_3D_update += 1
        time_since_last_3D_update += sim1D.dt
        
        if time_since_last_3D_update // update_lag_time > 0.0:
        # if num_events_since_last_3D_update % update_lag == 0 and time3D<sim1D.time:
            print("\nUpdate 3D sims ...")
            anchor_locations, activation_times, leids = add_new_LEFBonds(sim1D, sim3D, num_events_since_last_3D_update)
            
            assert anchor_locations.shape[0]==num_events_since_last_3D_update+1, f'shape mismatch!! {anchor_locations.shape,num_events_since_last_3D_update}'
                            
            activated_index = 0
            activate_LEFBonds(sim3D, anchor_locations[activated_index],kLE=lef.k_LE)
            # print(anchor_locations.shape)
            #output the anchors
            # lef.output_anchor_locations(output)
            lef.reset_LE_resisting_forces()
            
            for time_stamp in np.arange(time3D, sim1D.time, tblock):
                
                index_to_be_activated = closest_lower_index(time_stamp, activation_times)
                if index_to_be_activated is not None and index_to_be_activated != activated_index:
                    # if new_bonds_needed(anchor_locations[index_to_be_activated], anchor_locations[activated_index]):
                    if not np.array_equal(anchor_locations[index_to_be_activated], anchor_locations[activated_index]):
                        activate_LEFBonds(sim3D, anchor_locations[index_to_be_activated],kLE=lef.k_LE)
                        print(f" --> Activated 1D time point: {activation_times[index_to_be_activated]:<8.5f} | Current 3D time:{time3D:<8.5f}")
                        # print(anchor_locations[index_to_be_activated])
                    activated_index = index_to_be_activated
                    
                try:
                    sim3D.run(blocksize, report=True)
                    # sim3D.printForces()
                    # sim3D.saveStructure()
                    lef.update_LE_resisting_force(leids[activated_index], anchor_locations[activated_index])
                    
                    anchor_loc_dict[time_stamp] = anchor_locations[activated_index]
                    coords = sim3D.state.getPositions(asNumpy=True)
                    resisting_force.append(sim1D.lef.loop_extruder_dict["LE_0"]['resisting_force'])
                    active_force[0].append(sim1D.lef.loop_extruder_dict["LE_0"]['active_force_left'])
                    active_force[1].append(sim1D.lef.loop_extruder_dict["LE_0"]['active_force_right'])
                    
                    # print(time3D, sim1D.time, time_stamp, anchor_locations[activated_index])
                    le_bond_distance=np.linalg.norm(coords[anchor_locations[activated_index][0][0]]-coords[anchor_locations[activated_index][0][1]])
                    # print(coords.shape, anchor_locations[activated_index])
                    print(f'LE_bond--{le_bond_distance}', anchor_locations[activated_index][0][0], anchor_locations[activated_index][0][1])
                except ():
                    sim3D.print_force_info()
                time3D += tblock
                
            num_events_since_last_3D_update = 0
            time_since_last_3D_update = 0.0
            # lef.update_LE_resisting_forces()
            print("\nRun 1D sims ...")
            

In [10]:
run_sim(sim3D, sim1D, lef, tstop=1e3, update_interval=100.0, verbose=1)

2025-04-28 15:29:12,035 | WARNING | Forcefield | The system does not have force LEFBonds!!
2025-04-28 15:29:12,036 | WARNING | Forcefield | The forces applied in the system are: dict_keys(['HarmonicBonds', 'SelfAvoidance'])


traverse_LE: dx=2.1389697714804115 | force=4.868142630954384 | fres=(0.0, 0.0) | cutoff=0.9999546021312975, dt=0.43361147410498574, t_mesh=0.001
78 81.13896977148042
Time: 0.433611   |    LE_0    move_right_anchor    | dx_left: 0      | dx_right: 2     
Rate vector: [0.01 1.   1.   0.   0.  ]
LE_0 Stepped right
LE_0    currently anchored at  ( 78.00, 81.14) | left blocked by (None   ,None   ) | right blocked by (None   ,None   )
LE_0    previously anchored at ( 78.00, 79.00) | left blocked by (None   ,None   ) | right blocked by (None   ,None   )


traverse_LE: dx=1.0107576249761143 | force=5.408959133710566 | fres=(0.0, 0.0) | cutoff=0.9999546021312975, dt=0.19207027758033043, t_mesh=0.001
78 82.14972739645653
Time: 0.625682   |    LE_0    move_right_anchor    | dx_left: 0      | dx_right: 1     
Rate vector: [0.01 1.   1.   0.   0.  ]
LE_0 Stepped right
LE_0    currently anchored at  ( 78.00, 82.15) | left blocked by (None   ,None   ) | right blocked by (None   ,None   )
LE_0    prev

ValueError: 